In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!git clone https://huggingface.co/camenduru/Wav2Lip
!pip install yt_dlp ffmpeg-python librosa==0.8.0
!pip install edge-tts
!pip install firebase-admin

!pip install "numpy<1.24"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
fatal: destination path 'Wav2Lip' already exists and is not an empty directory.


In [ ]:

%cd Wav2Lip
import traceback
import firebase_admin
from firebase_admin import credentials, db
import time
import signal
import json
import uuid  # Import the uuid module
import subprocess

from base64 import b64encode

User = 'sekarc37' #@param {type:"string"}

# Initialize Firebase Admin SDK
cred = credentials.Certificate('/content/gdrive/MyDrive/avatars/firebase.json')
firebase_admin.initialize_app(cred, {'databaseURL': 'https://videogenerator-b1c1e-default-rtdb.firebaseio.com'})


/content/Wav2Lip


In [ ]:
#utility functions
import os
import shlex
import shutil

def generate_video(uuid,voice, question, answer, avatar, audio_person, text_bottom_to_overlay):
    try:
        video_dir = f"/content/outputs/{uuid}"
        os.makedirs(video_dir, exist_ok=True)
        question_path = lets_generate_video(uuid, voice, question, avatar, audio_person, is_question = True)
        answer_path = ""
        if answer:
            answer_path = lets_generate_video(uuid, voice, answer, avatar, audio_person, is_question = False)
        if question_path:
            return merge_videos(uuid, avatar,question,answer,text_bottom_to_overlay,question_path,answer_path)
        else:
            return ''
    except Exception as e:
        print(traceback.format_exc())
        return ''

def lets_generate_video(uuid, voice, input_text, avatar, audio_person, is_question):
    try:
        audio_output = generate_audio(uuid, voice, input_text, is_question)
        output_path = ''
        if audio_output:
          if 'elsa1' == avatar:
            !python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/gdrive/MyDrive/avatars/elsa1.mp4" --audio $audio_output
          if 'elsa2' == avatar:
            !python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/gdrive/MyDrive/avatars/elsa2.mp4" --audio $audio_output
          if 'elsa3' ==avatar:
            !python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/gdrive/MyDrive/avatars/elsa3.mp4" --audio $audio_output
          if 'hari1' == avatar:
            !python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/gdrive/MyDrive/avatars/hari1.mp4" --audio $audio_output
          if 'hari2' == avatar:
            !python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/gdrive/MyDrive/avatars/hari2.mp4" --audio $audio_output
          if 'alex1' == avatar:
            !python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/gdrive/MyDrive/avatars/alex1.mp4" --audio $audio_output
          if 'leela1' == avatar:
            !python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/gdrive/MyDrive/avatars/leela1.mp4" --audio $audio_output
          if 'lisa1' == avatar:
            !python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/gdrive/MyDrive/avatars/lisa1.mp4" --audio $audio_output

          # Process the video and get the output video file path
          wav2lip_output_video_path = "/content/Wav2Lip/results/result_voice.mp4"
          file_name = uuid+'_ans.mp4'

          video_dir = f"/content/outputs/{uuid}"
          if is_question:
            file_name = uuid+'_ques.mp4'
          output_path = video_dir+"/"+file_name

          shutil.copy(wav2lip_output_video_path, output_path)
        return output_path
    except Exception as e:
        print(traceback.format_exc())
        return ''

def merge_videos(id, avatar, question,answer,text_bottom_to_overlay,question_path,answer_path):
    try:
        video_dir = f"/content/outputs/{id}"
        output_video_path = f'{video_dir}/output_concatenated1.mp4'
        question_output_path = f'{video_dir}/out1.mp4'
        answer_output_path = f'{video_dir}/out2.mp4'

        question_box_color = "blue"
        answer_box_color = "green"
        question_output_path = add_text_overlay(question_path, question_output_path, question, text_bottom_to_overlay,question_box_color)
        if answer_path:
            answer_output_path = add_text_overlay(answer_path, answer_output_path, answer, text_bottom_to_overlay,answer_box_color)
            if answer_output_path:
                concatenate_videos([question_output_path, answer_output_path], output_video_path)
        else:
            output_video_path = question_output_path
        final_output = f'{video_dir}/{id}.mp4'
        # os.rename(output_video_path , final_output)
        shutil.copy(output_video_path, final_output)

        return final_output
    except Exception as e:
        print(traceback.format_exc())
        return ''


def add_text_overlay(input_video, output_video, text_top, text_bottom, box_color, font_size=36, font_color='black', margin_right_percent=96):
    # Initialize variables for processing text
    chars_per_line = 50  # Adjust based on your specific font and size
    lines_top = []
    current_line = ''

    # Process each word in the input text
    for word in text_top.split():
        # Check if adding the next word exceeds the character limit per line
        if len(current_line + ' ' + word) > chars_per_line:
            # If current line is full, add it to the list and start a new line
            lines_top.append(current_line)
            current_line = word
        else:
            # If not, add the word to the current line, include space if not the first word
            if current_line:
                current_line += ' '
            current_line += word

    # Add the last line if it contains any text
    if current_line:
        lines_top.append(current_line)

    # Join lines with line breaks to form the formatted text
    formatted_text_top = '\n'.join(lines_top)

    # Escape the formatted text for command line usage
    escaped_text_top = shlex.quote(formatted_text_top)

    # Set parameters for ffmpeg command
    right_margin = f"main_w*{margin_right_percent/100}-tw-10"
    left_margin = 460
    x_coordinate_top = left_margin
    bottom_margin = 74
    center_horizontal = "(main_w-text_w)/2"

    # Construct and run the ffmpeg command
    cmd = [
        'ffmpeg',
        '-y',
        '-i', input_video,
        '-vf', f'drawtext=text={escaped_text_top}:x={x_coordinate_top}:y=(h-text_h)/2:fontfile=/content/gdrive/MyDrive/avatars/LeagueSpartan-SemiBold.ttf:fontsize={font_size}:fontcolor={font_color}:box=1:boxcolor={box_color}@0.5:boxborderw=20,drawtext=text={shlex.quote(text_bottom)}:x={center_horizontal}:y=h-th-{bottom_margin}:fontfile=/content/gdrive/MyDrive/avatars/LeagueSpartan-SemiBold.ttf:fontsize={font_size*1.2}:fontcolor={font_color}',
        '-c:a', 'copy',
        output_video
    ]
    subprocess.run(cmd)
    return output_video

# def add_text_overlay(input_video, output_video, text_top, text_bottom,box_color, font_size=36, font_color='black', margin_right_percent=96):
#     words_top = text_top.split()
#     lines_top = []

#     # Combine words into lines with 8 words per line
#     for i in range(0, len(words_top), 7):
#         line_top = ' '.join(words_top[i:i+7])
#         lines_top.append(line_top)

#     # Join lines with line breaks
#     formatted_text_top = '\n'.join(lines_top)

#     escaped_text_top = shlex.quote(formatted_text_top)

#     # Calculate the right margin based on the video width and margin percentage
#     right_margin = f"main_w*{margin_right_percent/100}-tw-10"

#     # Adjust the x-coordinate to start from the left 400 pixels
#     left_margin = 470
#     x_coordinate_top = left_margin

#     # Calculate bottom margin and center horizontally
#     bottom_margin = 74
#     center_horizontal = "(main_w-text_w)/2"
#     cmd = [
#         'ffmpeg',
#         '-y',
#         '-i', input_video,
#         '-vf', f'drawtext=text={escaped_text_top}:x={x_coordinate_top}:y=(h-text_h)/2:fontfile=/content/gdrive/MyDrive/avatars/LeagueSpartan-SemiBold.ttf:fontsize={font_size}:fontcolor={font_color}:box=1:boxcolor={box_color}@0.5:boxborderw=20,drawtext=text={shlex.quote(text_bottom)}:x={center_horizontal}:y=h-th-{bottom_margin}:fontfile=/content/gdrive/MyDrive/avatars/LeagueSpartan-SemiBold.ttf:fontsize={font_size*1.2}:fontcolor={font_color}',
#         '-c:a', 'copy',
#         output_video
#     ]
#     subprocess.run(cmd)
#     return output_video

def concatenate_videos(input_videos, output_video):
  try:
    # Create a text file with the list of video files
    with open('video_list.txt', 'w') as file:
        for video in input_videos:
            file.write(f"file '{video}'\n")

    # Run FFmpeg to concatenate the videos
    cmd = [
        'ffmpeg',
        '-y',
        '-f', 'concat',
        '-safe', '0',  # Allow absolute paths in the concat file
        '-i', 'video_list.txt',
        '-c', 'copy',
        output_video
    ]

    subprocess.run(cmd)
    return output_video
  except Exception as e:
      print(e)
      return ''

def generate_audio(uuid, voice, content, is_question):
  try:
      file_name = "audio_ans.mp3"
      if is_question:
        file_name = "audio_ques.mp3"
      audio_path = f"/content/outputs/{uuid}/{file_name}"
      audio_vvt = f"/content/outputs/{uuid}/{file_name}"

      cmd = [
          "edge-tts", "--voice",voice,
          "--text", content,
          "--write-media", 'output.mp3',
          "--write-subtitles", 'output_vtt.mp3'
      ]

      try:
          # Use subprocess.Popen to allow waiting for the process to finish
          process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

          # Print output in real-time while the process is running
          for line in process.stdout:
              print(line.strip())

          # Wait for the process to finish
          process.wait()

          shutil.copy('output.mp3', audio_path)
          return audio_path
      except subprocess.CalledProcessError as e:
        print(e)
        return ''
      return ''
  except Exception as e:
      print(e)
      return ''


In [ ]:

import datetime
import requests


firebase_user_ref = db.reference(User)
firebase_master_queue_ref = db.reference('videogenerator')
firebase_system_nodes_ref = db.reference('system_nodes')

isExisiting = False
def updateStatuOfSystemKey(system_key, new_status):
    try:
        # Update the status
        firebase_system_nodes_ref.child(system_key).set(new_status)
        print(f"************ Status updated for {system_key} to {new_status} ************")
    except Exception as e:
        print(f"************ Status update failed for {system_key} to {new_status} ************")
        print(traceback.format_exc())

def updateStatus(record_uuid, status,output_video_url, message):  # Use UUID for updating status
    try:
        # Get the reference to the record in the master_queue
        record_ref = firebase_master_queue_ref.child(record_uuid)
        record_user_ref = firebase_user_ref.child(record_uuid)

        # Update the status to "added_to_queue"
        record_ref.child('status').set(status)
        record_user_ref.child('status').set(status)
        # Update the status to "added_to_queue"
        record_ref.child('message').set(message)
        record_user_ref.child('message').set(message)
        # Update the status to "added_to_queue"
        record_ref.child('output_video_url').set(output_video_url)
        record_user_ref.child('output_video_url').set(output_video_url)

        print(f"Status updated to 'added_to_queue' for record with UUID {record_uuid}")
    except Exception as e:
        print(traceback.format_exc())


def updateUtcTimeInFirebase(username):
    try:
        # Get the current UTC time
        current_utc_time = datetime.datetime.utcnow().isoformat()

        # Update the current time in the Firebase node
        firebase_system_nodes_ping_ref = db.reference(f'system_nodes_ping/{username}')
        firebase_system_nodes_ping_ref.set(current_utc_time)

        print(f"************ Updated UTC time for {username} to {current_utc_time} ************")
    except Exception as e:
        print(f"************ Failed to update UTC time for {username} ************")
        print(traceback.format_exc())


def handle_new_record(event):
    try:
      if event.data and not isExisiting:
        new_record = event.data
        assignToService(new_record)
    except Exception as e:
        print(traceback.format_exc())

def upload_video(video_file):
    url = f"https://importantnews.in/videogenerator/adt/upload.php"
    files = {'video': open(video_file, 'rb')}
    try:
        response = requests.post(url, files=files)
        print(response.text)
        return response.text
    except Exception as e:
        print(f"Error uploading video: {e}")
        return ""

def setInactiveUsers():
    try:
        # Get a reference to the system_nodes_ping node
        system_nodes_ping_ref = db.reference('system_nodes_ping')

        # Get the current UTC time
        current_utc_time = datetime.datetime.utcnow()

        # Iterate through all users in system_nodes_ping
        for user in system_nodes_ping_ref.get():
            print(user)
            try:
                # Get the timestamp from the system_nodes_ping node
                user_timestamp = datetime.datetime.fromisoformat(system_nodes_ping_ref.child(user).get())
                # Calculate the time difference
                time_difference = current_utc_time - user_timestamp
                # Check if the time difference is less than 30 seconds
                if time_difference.total_seconds() > 30:
                    # Set the corresponding /system_nodes/<User> to inactive
                    system_nodes_ref = db.reference(f'system_nodes/{user}')
                    system_nodes_ref.set('busy')
                    print(f"************ Set {user} to inactive ************")

            except Exception as e:
                print(f"************ Failed to process user {user} ************")
                print(traceback.format_exc())

    except Exception as e:
        print("************ Failed to retrieve system_nodes_ping data ************")
        print(traceback.format_exc())

def assignToService(new_record):
    try:
        isExisiting = True
        node_data = json.loads(json.dumps(new_record))
        if node_data:
          # Check if 'content' key exists in the response JSON
          if 'question' in node_data:
            # Access individual fields
            answer = node_data['answer']
            question = node_data['question']
            avatar = node_data['avatar']
            # language = node_data['language']
            # voice = node_data['voice']
            voice = "en-CA-ClaraNeural"

            title = node_data['title']
            id = str(node_data['id'])
            status = node_data['status']
            uuid = node_data['uuid']
            if status == "yet_to_process":
              updateStatuOfSystemKey(User, 'busy')
              updateStatus(uuid, 'in_progress', 'in_progress', 'in_progress')

              video = generate_video(uuid, voice, question, answer, avatar, voice, title)
              if video:
                urlresp = upload_video(video)
                updateStatus(uuid, 'completed',f"https://importantnews.in/videogenerator/adt/uploads/{uuid}.mp4", 'completed')
              else:
                updateStatus(uuid, 'error','error', 'error')

              updateStatuOfSystemKey(User, 'available')
          isExisiting = False
    except Exception as e:
        print(traceback.format_exc())
        updateStatuOfSystemKey(User, 'available')
        isExisiting = False
        try:
          if uuid:
            updateStatus(uuid, 'error', 'error', 'error')
        except Exception as e:
            pass


def getOneRecord():
    response_list = None
    try:
        # Get all nodes with status as "yet_to_process"
        nodes_to_process = firebase_master_queue_ref.order_by_child('status').equal_to('yet_to_process').get()
        # Sort nodes based on UUID in ascending order
        sorted_nodes = sorted(nodes_to_process.values(), key=lambda x: x['id'])
        # Print the entire node
        for value in sorted_nodes:
            return value
        return response_list
    except Exception as e:
        print(traceback.format_exc())
        return response_list

firebase_user_ref.listen(handle_new_record)

# Use a flag to gracefully exit the script
exit_flag = False

def signal_handler(sig, frame):
    global exit_flag
    print("Received interrupt signal. Exiting...")
    exit_flag = True

# Register the signal handler for SIGINT (Ctrl+C)
signal.signal(signal.SIGINT, signal_handler)

updateStatuOfSystemKey(User, 'available')
# Run the script until an interrupt signal is received
while not exit_flag:
    assignToService(getOneRecord())
    updateUtcTimeInFirebase(User)  # Update UTC time in Firebase for the current user
    setInactiveUsers()
    time.sleep(10)


************ Status updated for sekarc37 to available ************
************ Updated UTC time for sekarc37 to 2024-03-27T17:44:34.636235 ************
bhuvan1
************ Set bhuvan1 to inactive ************
saravananchandrasekaran1995
************ Set saravananchandrasekaran1995 to inactive ************
saravananchandrasekaran95
************ Set saravananchandrasekaran95 to inactive ************
sekarc37
************ Updated UTC time for sekarc37 to 2024-03-27T17:44:45.315922 ************
bhuvan1
************ Set bhuvan1 to inactive ************
saravananchandrasekaran1995
************ Set saravananchandrasekaran1995 to inactive ************
saravananchandrasekaran95
************ Set saravananchandrasekaran95 to inactive ************
sekarc37
************ Updated UTC time for sekarc37 to 2024-03-27T17:44:56.362032 ************
bhuvan1
************ Set bhuvan1 to inactive ************
saravananchandrasekaran1995
************ Set saravananchandrasekaran1995 to inactive ************
s

In [ ]:
# import subprocess

# # Define paths to your video and subtitle files, and the output file
# input_video = 'path_to_your_video.mp4'
# subtitle_file = 'path_to_your_subtitle.vtt'
# output_video = 'path_to_output_video.mp4'

# # Example usage
# input_video = '/content/outputs/485d8cec-0390-4215-bc47-164f7850866e/485d8cec-0390-4215-bc47-164f7850866e_ques.mp4'
# output_video = '/content/outputs/485d8cec-0390-4215-bc47-164f7850866e/485d8cec-0390-4215-bc47-164f7850866e_ques_out.mp4'
# subtitle_file = '/content/outputs/485d8cec-0390-4215-bc47-164f7850866e/output.vtt'


# # FFmpeg command with subtitles centered
# # 'Alignment' is set to 2 for center alignment in ASS subtitles. We also set 'MarginV' to control vertical margin.
# # Note: FFmpeg converts VTT to ASS format for processing. 'Alignment=2' is for top-centered (use '8' for middle-centered).
# # Adjust 'MarginV' value to position subtitles vertically as desired. Smaller values bring text lower.
# command = f'ffmpeg -y -i {input_video} -vf "subtitles={subtitle_file}:force_style=\'Alignment=8,MarginV=20\'" {output_video}'

# # Execute the command
# try:
#     subprocess.run(command, check=True, shell=True)
#     print(f'Subtitles were successfully burned into {output_video}')
# except subprocess.CalledProcessError as e:
#     print(f'An error occurred: {e}')

